In [1]:
! pip install transformers
! pip install scipy
! pip install vaderSentiment
! pip install deep-translator
! pip install emoji
! pip install contractions
! pip install nltk
! pip install spacy
! python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


working on this now

In [2]:
import subprocess
import sys

# Function to install packages
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install necessary packages
install('vaderSentiment')
install('pandas')
install('nltk')
install('transformers')
install('scipy')
install('deep-translator')
install('emoji')
install('contractions')

import pandas as pd
import numpy as np
import nltk
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from scipy.special import softmax
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import string
import contractions
import emoji
from deep_translator import GoogleTranslator

# Download and initialize resources
nltk.download('vader_lexicon')
vader_analyzer = SentimentIntensityAnalyzer()
tokenizer = RobertaTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest')
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest')

# Read the CSV file
file_path = "lifestyle_reviews 1.csv"
df = pd.read_csv(file_path)

# Select specific columns
df = df[["content", "score", "thumbsUpCount", "reviewCreatedVersion", "repliedAt", "at", "replyContent", "repliedAt", "appVersion"]]

# Convert data types
df["score"] = df["score"].astype(int)
df["thumbsUpCount"] = df["thumbsUpCount"].astype(int)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with

In [3]:

# Define function for text processing
def clean_text(text):
    if text:
        text = str(text).lower()
        text = contractions.fix(text)
        text = re.sub(r"@\w+", " at ", text)
        text = re.sub(r"#(\w+)", r'\1', text)
        text = re.sub(r"http\S+|www\S+", "", text)
        text = re.sub(r"<.*?>", "", text)
        text = text.translate(str.maketrans('', '', string.punctuation.replace('!', '').replace('?', '')))
        text = emoji.demojize(text)
        text = re.sub(r'\[.*?\]', '', text)
        text = re.sub(r'\s+', ' ', text.strip())
        text = re.sub(r'\n', '', text)
        text = re.sub(r'\w*\d\w*', '', text)
        text = re.sub(r'[^\x00-\x7F]+', '', text)
        text = " ".join(text.split())
    return text

# Apply the text cleaning function to the 'content' column
df['clean_content'] = df['content'].apply(clean_text)

# Filter out rows where 'clean_content' is NaN or an empty string
df = df[df['clean_content'].notna() & (df['clean_content'] != "")]





In [4]:
# Define the function for translating to English
def translate_to_english(text):
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except Exception as e:
        return text

# Apply the translation function to the 'clean_content' column
df['translated_content'] = df['clean_content'].apply(translate_to_english)

In [6]:
import pandas as pd
import numpy as np
import re

# Define categories and synonyms
categories = [
    ('app', 'technical'),
    ('app', 'user_interface'),
    ('app', None),
    ('product', None),
    ('coupons_and_discounts', None),
    ('price', None),
    ('service', None),
    ('returns_and_refunds', None),
    ('store_operations', None),
    ('delivery', None)
]

synonyms = {
    'app': [
        'app', 'checkout', 'performance','software', 'program', 'tool', 'system', 'platform', 'application', 'utility', 'module',
        'feature', 'applet', 'widget', 'mobile', 'desktop', 'web',  'digital app', 'payment', 'features'
    ],
    'technical': [
        'technical problems', 'error', 'stuck', 'errors', 'glitches', 'failures', 'malfunctions',
        'bugs', 'server', 'loading', 'sign in', 'signin','reinstall','install', 'login', 'logout', 'otp', 'OTP', 'crash','gateway'
    ],
    'user_interface': [
        'UI', 'user experience', 'interface', 'screen', 'layout', 'user interaction', 'display', 'user panel', 'controls',
        'user view', 'navigation', 'visuals', 'graphics', 'elements', 'dashboard', 'interaction', 'menu',
        'usability', 'workflow', 'components', 'touchpoints', 'graphic design', 'user-centered',
        'interface design', 'user interface', 'user-facing'
    ],
    'product': [
        'item', 'apparel', 'styles','style', 'collection', 'fit', 'merchandise', 'commodity',
        'material', 'offering', 'product line', 'product range', 'product set', 'color', 'colour',
        'product model', 'product type', 'unit', 'category', 'variant',
        'variety', 'fancy', 'clothing', 'designs', 'stylish', 'eco-friendly', 'collections', 'varieties', 'materials', 'durable',
        'beautiful', 'trendy', 'accessories', 'fashionable', 'fabrics', 'selection', 'colourful',
        'shoes', 'clothes', 'shirts', 'jeans', 'tshirts', 'pants', 'suits', 'accessory', 'wardrobe'
    ],
    'coupons_and_discounts': [
        'discount', 'coupon', 'deal', 'promotion', 'offer', 'sale', 'rebate', 'voucher', 'markdown', 'savings', 'bargain',
        'price cut', 'price drop', 'price reduction', 'special'
    ],
    'price': [
        'price','pricy', 'pricey', 'competitive prices', 'reasonable price',
        'affordable', 'cheap', 'cheapest', 'great prices', 'rates', 'budget', 'cashback',
        'bumper', 'rewards', 'expensive', 'pay', 'charges', 'paying', 'value', 'valued', 'costly' ,'cost', 'rate', 'charge', 'fee', 'expense', 'amount', 'pricing',
        'bill', 'figure', 'quote', 'bargain'
    ],
    'service': [
        'service', 'support', 'assistance', 'help', 'aid', 'customer service', 'customer support'
    ],
    'returns_and_refunds': [
        'return', 'send back', 'return item', 'return product', 'return merchandise', 'return goods', 'return shipment', 'product return',
        'return order', 'return request', 'item return', 'goods return', 'merchandise return', 'return process', 'return policy',
        'return procedure', 'return authorization', 'return package', 'return shipment', 'return reason', 'return label', 'return slip',
        'return receipt', 'return form', 'return notice', 'return instruction', 'return guideline', 'return information', 'return conditions',
        'return terms', 'return rules', 'return timeframe', 'return period', 'return allowance', 'return option', 'return window', 'return deadline',
        'return authorization number', 'return shipping', 'return status', 'return tracking', 'return account', 'return initiation', 'return query',
        'return note', 'return case', 'return charge', 'return fee', 'return refund', 'return repayment', 'return reimbursement',
        'refund', 'repayment', 'reimbursement', 'money back', 'cash back', 'credit', 'refund amount', 'refund request', 'refund policy',
        'refund process', 'refund procedure', 'refund authorization', 'refund transaction', 'refund form', 'refund slip', 'refund receipt',
        'refund confirmation', 'refund approval', 'refund status', 'refund tracking', 'refund account', 'refund initiation', 'refund notice',
        'refund information', 'refund guideline', 'refund terms', 'refund conditions', 'refund rules', 'refund period', 'refund window',
        'refund deadline', 'refund charge', 'refund fee', 'refund inquiry', 'refund query', 'refund case', 'refund claim', 'refund process',
        'refund arrangement', 'refund management', 'refund order', 'refund application', 'refund note', 'refund explanation', 'refund details',
        'refund notification', 'refund alert', 'refund notice'
    ],
    'store_operations': [
        'store', 'shop', 'operations', 'staff'
    ],
    'delivery': [
        'delivery', 'quick','fast','late','delay','cancelled', 'pickup', 'collect','shipped','track','partner',
        'courier','delayed','delay','destination','pick-up','bluedart','xpressbees','transit',
        'parcel','package', 'shipping', 'shipment', 'dispatch',
        'distribution', 'transport','deliver','delivered','delivering'
    ]
}

# Function to categorize content
def categorize_content(content):
    matched_categories = []
    matched_subcategories = []

    for category, subcategory in categories:
        category_matched = False
        if category in synonyms:
            for synonym in synonyms[category]:
                if re.search(r'\b' + re.escape(synonym) + r'\b', content, re.IGNORECASE):
                    if category not in matched_categories:
                        matched_categories.append(category)
                    category_matched = True

        if category_matched and subcategory and subcategory in synonyms:
            for synonym in synonyms[subcategory]:
                if re.search(r'\b' + re.escape(synonym) + r'\b', content, re.IGNORECASE):
                    if subcategory not in matched_subcategories:
                        matched_subcategories.append(subcategory)
        elif category_matched and subcategory is None:
            matched_subcategories.append('')

    # Remove empty strings from matched_subcategories
    matched_subcategories = [sub for sub in matched_subcategories if sub]

    return ','.join(matched_categories), ','.join(matched_subcategories)

In [9]:
# VADER sentiment analysis function
def vader_sentiment_analysis(text):
    sentiment_scores = vader_analyzer.polarity_scores(text)
    return sentiment_scores

def roberta_sentiment_analysis(text):
    try:
        # Tokenize and truncate the text to the maximum length allowed by the model
        encoded_text = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)

        # Perform sentiment analysis
        output = model(**encoded_text)
        scores = output.logits[0].detach().numpy()

        # Apply softmax to get probabilities
        scores = softmax(scores)
        scores_dict = {
            'roberta_neg': float(scores[0]),
            'roberta_neu': float(scores[1]),
            'roberta_pos': float(scores[2])
        }
        return scores_dict
    except Exception as e:
        # Log or print the error message
        print(f"Error processing text: {text}\nError: {e}")
        return {'roberta_neg': 0, 'roberta_neu': 0, 'roberta_pos': 0}

def process_dataframe(df, text_column):
    # VADER sentiment analysis
    vader_results = df[text_column].apply(lambda x: vader_sentiment_analysis(x) if pd.notnull(x) else np.nan)
    df['vader_neg'] = vader_results.apply(lambda x: round(x['neg'] * 100, 2) if pd.notnull(x) else np.nan)
    df['vader_neu'] = vader_results.apply(lambda x: round(x['neu'] * 100, 2) if pd.notnull(x) else np.nan)
    df['vader_pos'] = vader_results.apply(lambda x: round(x['pos'] * 100, 2) if pd.notnull(x) else np.nan)
    df['vader_compound'] = vader_results.apply(lambda x: round(x['compound'], 2) if pd.notnull(x) else np.nan)

    # RoBERTa sentiment analysis
    roberta_results = df[text_column].apply(lambda x: roberta_sentiment_analysis(x) if pd.notnull(x) else {'roberta_neg': 0, 'roberta_neu': 0, 'roberta_pos': 0})
    df['roberta_neg'] = roberta_results.apply(lambda x: round(x['roberta_neg'] * 100, 2) if pd.notnull(x) else np.nan)
    df['roberta_neu'] = roberta_results.apply(lambda x: round(x['roberta_neu'] * 100, 2) if pd.notnull(x) else np.nan)
    df['roberta_pos'] = roberta_results.apply(lambda x: round(x['roberta_pos'] * 100, 2) if pd.notnull(x) else np.nan)

     # Categorize content
    categories_and_subcategories = df[text_column].apply(lambda x: categorize_content(x) if pd.notnull(x) else ('', ''))
    df['Categories'], df['Subcategories'] = zip(*categories_and_subcategories)

    # Calculate weighted averages
    df['weighted_neg'] = round((df['vader_neg'].fillna(0) + df['roberta_neg'].fillna(0)) / 2, 0)
    df['weighted_neu'] = round((df['vader_neu'].fillna(0) + df['roberta_neu'].fillna(0)) / 2, 0)
    df['weighted_pos'] = round((df['vader_pos'].fillna(0) + df['roberta_pos'].fillna(0)) / 2, 0)

    # Define classify_likert function
    def classify_likert(vader_pos, roberta_pos, weighted_pos, weighted_neu, weighted_neg, roberta_neg, vader_compound, vader_neu, roberta_neu, vader_neg):
        max_pos = max(vader_pos, roberta_pos, weighted_pos)
        max_neu = max(vader_neu, roberta_neu, weighted_neu)
        max_neg = max(vader_neg, roberta_neg, weighted_neg)
        if max_pos > 80:
            return "Very Satisfied"
        elif max_pos > 60 and max_pos <= 80:
            return "Satisfied"
        elif max_neu > 40 and max_neu <= 60:
            return "Neutral"
        elif max_neg > 60 and max_neg <= 80:
            return "Dissatisfied"
        elif max_neg > 80:
            return "Very Dissatisfied"
        else:
            return "Mixed Feelings"

    # Apply classify_likert function
    df['likert_scale'] = df.apply(lambda row: classify_likert(
        row['vader_pos'], row['roberta_pos'], row['weighted_pos'], row['weighted_neu'], row['weighted_neg'],
        row['roberta_neg'], row['vader_compound'], row['vader_neu'], row['roberta_neu'], row['vader_neg']
    ), axis=1)

    # Select relevant columns for the final DataFrame
    result_df = df[[
        'score',
        'thumbsUpCount',
        'reviewCreatedVersion',
        'at',
        'repliedAt',
        'clean_content',
        'translated_content',
        'weighted_neg',
        'weighted_neu',
        'weighted_pos',
        'Categories',
        'Subcategories',
        'likert_scale'
    ]]

    return result_df



In [10]:

# Apply the process_dataframe function
results_df = process_dataframe(df, "translated_content")
results_df = results_df.dropna()

# Save the results to a CSV file
results_df.to_csv("final_results.csv", index=False)

# Display the results
print(results_df.head())

    score  thumbsUpCount reviewCreatedVersion                at  \
4       5              0                  8.7  23-07-2024 05:27   
7       1              0                 6.36  22-07-2024 12:45   
9       1              0                  8.7  22-07-2024 05:12   
11      1              0                8.7.2  21-07-2024 12:25   
12      1              0                8.7.2  21-07-2024 10:58   

           repliedAt         repliedAt  \
4   23-07-2024 06:22  23-07-2024 06:22   
7   23-07-2024 06:21  23-07-2024 06:21   
9   23-07-2024 09:34  23-07-2024 09:34   
11  23-07-2024 09:33  23-07-2024 09:33   
12  22-07-2024 06:27  22-07-2024 06:27   

                                        clean_content  \
4                                            best app   
7                        support service is the worst   
9   very horrible experiencethey even do not deliv...   
11  useless application uses your money and after ...   
12  very bad high price wala me :face_with_symbols...   

 